In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(174, size1)
        self.conv2 = GCNConv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [3]:
for rs in range(10):
    
    # read data
    scaler = MinMaxScaler()
    X_train = pd.read_csv("../../result/input_GTEx_phastcons/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_train = scaler.fit_transform(X_train)
    X_valid = pd.read_csv("../../result/input_GTEx_phastcons/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_valid = scaler.transform(X_valid)
    X_test = pd.read_csv("../../result/input_GTEx_phastcons/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    X_test = scaler.transform(X_test)
    Y_train = pd.read_csv("../../result/input_GTEx_phastcons/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_GTEx_phastcons/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_GTEx_phastcons/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_GTEx_phastcons/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_GTEx_phastcons/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_GTEx_phastcons/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_threshold0.3", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_GM12878", "CoExpr_ENCODE_GM12878_0.75", "CoExpr_GTEx_EBV_0.75", "Random_ER", "Random_SF"]:
        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_GTEx_phastcons/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_GTEx_phastcons/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_GTEx_phastcons/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-10 12:48:05,716] A new study created in memory with name: no-name-8857cdc7-75b4-4ae1-b0a2-b53466ed3c64
[I 2023-12-10 12:48:10,984] Trial 0 finished with value: 0.00919300876557827 and parameters: {'size1': 871, 'size2': 246, 'size3': 454, 'num_epochs': 13, 'lr': 0.00034995397045677203}. Best is trial 0 with value: 0.00919300876557827.
[I 2023-12-10 12:48:11,699] Trial 1 finished with value: 0.010536763817071915 and parameters: {'size1': 231, 'size2': 430, 'size3': 199, 'num_epochs': 157, 'lr': 0.008144229309025398}. Best is trial 0 with value: 0.00919300876557827.
[I 2023-12-10 12:48:12,389] Trial 2 finished with value: 0.00914444588124752 and parameters: {'size1': 732, 'size2': 279, 'size3': 373, 'num_epochs': 121, 'lr': 0.00029581350720553107}. Best is trial 2 with value: 0.00914444588124752.
[I 2023-12-10 12:48:12,695] Trial 3 finished with value: 0.009418700821697712 and parameters: {'size1': 643, 'size2': 427, 'size3': 61, 'num_epochs': 52, 'lr': 0.006484159575468515}. 

[I 2023-12-10 12:48:36,810] Trial 4 finished with value: 0.009686141274869442 and parameters: {'size1': 139, 'size2': 509, 'size3': 80, 'num_epochs': 6, 'lr': 0.002972809169906293}. Best is trial 0 with value: 0.009264292195439339.
[I 2023-12-10 12:48:36,850] Trial 5 pruned. 
[I 2023-12-10 12:48:36,870] Trial 6 pruned. 
[I 2023-12-10 12:48:36,920] Trial 7 pruned. 
[I 2023-12-10 12:48:36,947] Trial 8 pruned. 
[I 2023-12-10 12:48:36,974] Trial 9 pruned. 
[I 2023-12-10 12:48:37,040] Trial 10 pruned. 
[I 2023-12-10 12:48:37,086] Trial 11 finished with value: 0.11326248198747635 and parameters: {'size1': 47, 'size2': 506, 'size3': 13, 'num_epochs': 1, 'lr': 0.0028429404384590503}. Best is trial 0 with value: 0.009264292195439339.
[I 2023-12-10 12:48:37,269] Trial 12 pruned. 
[I 2023-12-10 12:48:37,327] Trial 13 pruned. 
[I 2023-12-10 12:48:38,479] Trial 14 finished with value: 0.010571010410785675 and parameters: {'size1': 240, 'size2': 432, 'size3': 146, 'num_epochs': 116, 'lr': 0.00290433

[I 2023-12-10 12:48:57,276] Trial 5 pruned. 
[I 2023-12-10 12:48:57,298] Trial 6 pruned. 
[I 2023-12-10 12:48:57,319] Trial 7 pruned. 
[I 2023-12-10 12:48:57,355] Trial 8 pruned. 
[I 2023-12-10 12:48:57,379] Trial 9 pruned. 
[I 2023-12-10 12:48:57,449] Trial 10 pruned. 
[I 2023-12-10 12:48:57,509] Trial 11 pruned. 
[I 2023-12-10 12:48:57,571] Trial 12 pruned. 
[I 2023-12-10 12:48:57,632] Trial 13 pruned. 
[I 2023-12-10 12:48:57,695] Trial 14 pruned. 
[I 2023-12-10 12:48:57,749] Trial 15 pruned. 
[I 2023-12-10 12:48:57,823] Trial 16 pruned. 
[I 2023-12-10 12:48:58,041] Trial 17 pruned. 
[I 2023-12-10 12:48:58,094] Trial 18 pruned. 
[I 2023-12-10 12:48:58,161] Trial 19 pruned. 
[I 2023-12-10 12:48:58,570] A new study created in memory with name: no-name-e4fb2125-c579-4745-88b9-2b4602129c72
[I 2023-12-10 12:48:59,105] Trial 0 finished with value: 0.009225410409271717 and parameters: {'size1': 117, 'size2': 188, 'size3': 307, 'num_epochs': 67, 'lr': 0.00722454743546035}. Best is trial 0 wi

[I 2023-12-10 12:49:25,990] Trial 4 finished with value: 0.013136208057403564 and parameters: {'size1': 326, 'size2': 334, 'size3': 23, 'num_epochs': 45, 'lr': 0.008502808948761409}. Best is trial 4 with value: 0.013136208057403564.
[I 2023-12-10 12:49:26,010] Trial 5 pruned. 
[I 2023-12-10 12:49:26,036] Trial 6 pruned. 
[I 2023-12-10 12:49:26,064] Trial 7 pruned. 
[I 2023-12-10 12:49:26,083] Trial 8 pruned. 
[I 2023-12-10 12:49:28,047] Trial 9 finished with value: 0.019443484023213387 and parameters: {'size1': 543, 'size2': 39, 'size3': 298, 'num_epochs': 200, 'lr': 0.0012962553110686683}. Best is trial 4 with value: 0.013136208057403564.
[I 2023-12-10 12:49:29,225] Trial 10 finished with value: 0.014512777328491211 and parameters: {'size1': 37, 'size2': 193, 'size3': 42, 'num_epochs': 141, 'lr': 0.00701007910923573}. Best is trial 4 with value: 0.013136208057403564.
[I 2023-12-10 12:49:29,281] Trial 11 pruned. 
[I 2023-12-10 12:49:29,344] Trial 12 pruned. 
[I 2023-12-10 12:49:29,404]

[I 2023-12-10 12:49:57,267] Trial 9 pruned. 
[I 2023-12-10 12:49:57,318] Trial 10 pruned. 
[I 2023-12-10 12:49:57,367] Trial 11 pruned. 
[I 2023-12-10 12:49:57,423] Trial 12 pruned. 
[I 2023-12-10 12:49:57,475] Trial 13 pruned. 
[I 2023-12-10 12:49:57,527] Trial 14 pruned. 
[I 2023-12-10 12:49:57,581] Trial 15 pruned. 
[I 2023-12-10 12:49:57,639] Trial 16 pruned. 
[I 2023-12-10 12:49:57,690] Trial 17 pruned. 
[I 2023-12-10 12:49:57,743] Trial 18 pruned. 
[I 2023-12-10 12:49:58,869] Trial 19 finished with value: 0.014648833312094212 and parameters: {'size1': 226, 'size2': 152, 'size3': 165, 'num_epochs': 131, 'lr': 0.002329539722055192}. Best is trial 8 with value: 0.013157692737877369.
[I 2023-12-10 12:49:59,118] A new study created in memory with name: no-name-177be2ab-e0c7-4a04-8cce-6bd4b539308a
[I 2023-12-10 12:50:00,872] Trial 0 finished with value: 0.013027084060013294 and parameters: {'size1': 669, 'size2': 438, 'size3': 332, 'num_epochs': 140, 'lr': 0.007507903029585402}. Best i

...

[I 2023-12-10 12:59:34,108] Trial 4 finished with value: 0.016266457736492157 and parameters: {'size1': 326, 'size2': 247, 'size3': 147, 'num_epochs': 42, 'lr': 0.006888291772332589}. Best is trial 0 with value: 0.016247965395450592.
[I 2023-12-10 12:59:34,134] Trial 5 pruned. 
[I 2023-12-10 12:59:35,033] Trial 6 finished with value: 0.01618867553770542 and parameters: {'size1': 132, 'size2': 110, 'size3': 263, 'num_epochs': 109, 'lr': 0.006307350537036106}. Best is trial 6 with value: 0.01618867553770542.
[I 2023-12-10 12:59:35,072] Trial 7 pruned. 
[I 2023-12-10 12:59:35,096] Trial 8 pruned. 
[I 2023-12-10 12:59:35,607] Trial 9 finished with value: 0.016203461214900017 and parameters: {'size1': 421, 'size2': 150, 'size3': 28, 'num_epochs': 53, 'lr': 0.0022344260613075917}. Best is trial 6 with value: 0.01618867553770542.
[I 2023-12-10 12:59:35,737] Trial 10 pruned. 
[I 2023-12-10 12:59:35,796] Trial 11 pruned. 
[I 2023-12-10 12:59:35,861] Trial 12 pruned. 
[I 2023-12-10 12:59:35,927]

[I 2023-12-10 13:00:04,176] Trial 10 pruned. 
[I 2023-12-10 13:00:04,241] Trial 11 pruned. 
[I 2023-12-10 13:00:04,314] Trial 12 pruned. 
[I 2023-12-10 13:00:04,389] Trial 13 pruned. 
[I 2023-12-10 13:00:04,451] Trial 14 pruned. 
[I 2023-12-10 13:00:05,773] Trial 15 finished with value: 0.015842175111174583 and parameters: {'size1': 393, 'size2': 409, 'size3': 142, 'num_epochs': 121, 'lr': 0.0022871355417452676}. Best is trial 15 with value: 0.015842175111174583.
[I 2023-12-10 13:00:05,838] Trial 16 pruned. 
[I 2023-12-10 13:00:05,896] Trial 17 pruned. 
[I 2023-12-10 13:00:05,962] Trial 18 pruned. 
[I 2023-12-10 13:00:06,254] Trial 19 pruned. 
[I 2023-12-10 13:00:07,042] A new study created in memory with name: no-name-b699e2dc-66c4-432b-bead-1bd374ea22a4
[I 2023-12-10 13:00:09,153] Trial 0 finished with value: 0.015086113475263119 and parameters: {'size1': 633, 'size2': 325, 'size3': 216, 'num_epochs': 186, 'lr': 0.0048915008894226085}. Best is trial 0 with value: 0.015086113475263119

[I 2023-12-10 13:00:40,900] Trial 2 finished with value: 0.016226813197135925 and parameters: {'size1': 234, 'size2': 83, 'size3': 417, 'num_epochs': 69, 'lr': 0.009131189290069626}. Best is trial 2 with value: 0.016226813197135925.
[I 2023-12-10 13:00:42,151] Trial 3 finished with value: 0.016088109463453293 and parameters: {'size1': 202, 'size2': 391, 'size3': 410, 'num_epochs': 136, 'lr': 0.005758494308586801}. Best is trial 3 with value: 0.016088109463453293.
[I 2023-12-10 13:00:42,533] Trial 4 finished with value: 0.0162347499281168 and parameters: {'size1': 810, 'size2': 35, 'size3': 314, 'num_epochs': 33, 'lr': 0.0011981455535750635}. Best is trial 3 with value: 0.016088109463453293.
[I 2023-12-10 13:00:42,549] Trial 5 pruned. 
[I 2023-12-10 13:00:42,563] Trial 6 pruned. 
[I 2023-12-10 13:00:42,601] Trial 7 pruned. 
[I 2023-12-10 13:00:42,627] Trial 8 pruned. 
[I 2023-12-10 13:00:42,648] Trial 9 pruned. 
[I 2023-12-10 13:00:42,704] Trial 10 pruned. 
[I 2023-12-10 13:00:42,764] T

[I 2023-12-10 13:01:05,788] A new study created in memory with name: no-name-bca67e64-277e-47b4-a75f-89f05f82b4e0
[I 2023-12-10 13:01:06,156] Trial 0 finished with value: 0.029952896758913994 and parameters: {'size1': 718, 'size2': 258, 'size3': 62, 'num_epochs': 32, 'lr': 0.00017162628878803006}. Best is trial 0 with value: 0.029952896758913994.
[I 2023-12-10 13:01:07,904] Trial 1 finished with value: 0.014171123504638672 and parameters: {'size1': 59, 'size2': 431, 'size3': 306, 'num_epochs': 200, 'lr': 0.0015608501351415836}. Best is trial 1 with value: 0.014171123504638672.
[I 2023-12-10 13:01:09,361] Trial 2 finished with value: 0.015568277798593044 and parameters: {'size1': 45, 'size2': 398, 'size3': 245, 'num_epochs': 169, 'lr': 0.007662159017204174}. Best is trial 1 with value: 0.014171123504638672.
[I 2023-12-10 13:01:10,269] Trial 3 finished with value: 0.01447093952447176 and parameters: {'size1': 755, 'size2': 141, 'size3': 458, 'num_epochs': 82, 'lr': 0.0050547039018996745}

[I 2023-12-10 13:01:33,927] Trial 6 pruned. 
[I 2023-12-10 13:01:34,038] Trial 7 finished with value: 0.014744934625923634 and parameters: {'size1': 495, 'size2': 55, 'size3': 73, 'num_epochs': 23, 'lr': 0.0008329666566482087}. Best is trial 2 with value: 0.013487053103744984.
[I 2023-12-10 13:01:34,051] Trial 8 pruned. 
[I 2023-12-10 13:01:34,066] Trial 9 pruned. 
[I 2023-12-10 13:01:34,100] Trial 10 pruned. 
[I 2023-12-10 13:01:34,130] Trial 11 pruned. 
[I 2023-12-10 13:01:34,161] Trial 12 pruned. 
[I 2023-12-10 13:01:34,190] Trial 13 pruned. 
[I 2023-12-10 13:01:34,231] Trial 14 pruned. 
[I 2023-12-10 13:01:34,262] Trial 15 pruned. 
[I 2023-12-10 13:01:34,301] Trial 16 pruned. 
[I 2023-12-10 13:01:34,333] Trial 17 pruned. 
[I 2023-12-10 13:01:34,368] Trial 18 pruned. 
[I 2023-12-10 13:01:34,398] Trial 19 pruned. 
[I 2023-12-10 13:01:34,899] A new study created in memory with name: no-name-19ca5f36-32ac-4599-a591-d6ac9d4f99ab
[I 2023-12-10 13:01:35,616] Trial 0 finished with value: 0